# Minio create bucket
- open-source object storage server that stores unstructured data
- alternative to AWS S3 buckets
- AWS-S3 compatable
- python API ref: https://docs.minio.io/docs/python-client-api-reference.html
- to quickly set-up a server see: https://docs.minio.io/docs/minio-docker-quickstart-guide
    - easy to set-up on linux or windows
- in another notebook, we will try to use minio to store model artifacts for MLflow

## Import dependencies

In [1]:
from minio import Minio
from minio.error import ResponseError
import json
import os

## Instantiate a client object
- read envirnoment varaibles (should set them before open notebook)

In [2]:
minioClient = Minio('minio:9000', access_key='admin', secret_key='password', secure=False)

## List the buckets in the server
- none were found

In [3]:
minioClient.list_buckets()

## Let's create an s3 compatible bucket
- we'll name the bucket 'mlflow'

In [4]:
try:
    minioClient.make_bucket('mlflow')
except ResponseError as err:
    print(err)

BucketAlreadyOwnedByYou: BucketAlreadyOwnedByYou: message: Your previous request to create the named bucket succeeded and you already own it.

In [5]:
buckets = minioClient.list_buckets()
for bucket in buckets:
    print(bucket.name, bucket.creation_date)

data 2019-10-19 05:56:09.848000+00:00
drions 2019-10-19 05:55:45.812000+00:00
drive 2019-10-19 04:07:18.402000+00:00
mias 2019-10-19 04:08:53.466000+00:00
mlflow 2019-10-19 03:58:15.468000+00:00
mura 2019-10-19 04:07:22.694000+00:00


## Now we need to set the bucket policy
- see: https://docs.aws.amazon.com/AmazonS3/latest/dev/using-with-s3-actions.html

In [6]:
policy = {"Version":"2012-10-17",
        "Statement":[
            {
            "Sid":"",
            "Effect":"Allow",
            "Principal":{"AWS":"*"},
            "Action":"s3:GetBucketLocation",
            "Resource":"arn:aws:s3:::mlflow"
            },
            {
            "Sid":"",
            "Effect":"Allow",
            "Principal":{"AWS":"*"},
            "Action":"s3:ListBucket",
            "Resource":"arn:aws:s3:::mlflow"
            },
            {
            "Sid":"",
            "Effect":"Allow",
            "Principal":{"AWS":"*"},
            "Action":"s3:GetObject",
            "Resource":"arn:aws:s3:::mlflow/*"
            },
            {
            "Sid":"",
            "Effect":"Allow",
            "Principal":{"AWS":"*"},
            "Action":"s3:PutObject",
            "Resource":"arn:aws:s3:::mlflow/*"
            }

        ]}

In [7]:
minioClient.set_bucket_policy('mlflow', json.dumps(policy))

## List all the objects in a bucket

In [8]:
# List all object paths in bucket that begin with my-prefixname.
objects = minioClient.list_objects('mlflow', prefix='my',
                              recursive=True)
for obj in objects:
    print(obj.bucket_name, obj.object_name.encode('utf-8'), obj.last_modified,
          obj.etag, obj.size, obj.content_type)